<a href="https://colab.research.google.com/github/HairuoGao029/MasterThesis/blob/main/ARIMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/MSc/dcrnn

In [ ]:
from __future__ import print_function
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

data_raw = np.load('/content/drive/MyDrive/MSc/datasets/0810/8h_d20.npy', allow_pickle=True)
lane_list = np.load('/content/drive/MyDrive/MSc/net/lane_list.npy', allow_pickle=True)
id = ['56250003#0_0%1', '237111029#1.254_0%1', '238459506.6_0%1', '62830645#2.0.0_0%1'] # road segments for different attack scenarios
lane_id = '62830645#2.0.0_0%1'
lane_index = lane_list.tolist().index(lane_id)
print(len(data_raw[1][1]))
datar = data_raw[lane_index][1]
data = pd.Series(datar)
'''
discfile = 'E:/data_test.xls'
data = pd.read_excel(discfile,index_col=0)
data=data['number']
data.head()
'''

plt.plot(data)


diff_1 = data.diff(1)
diff1 = diff_1.dropna()
diff1_144_1 = diff_1-diff_1.shift(144)
diff1_144 = diff1_144_1.dropna()
#print(diff1_144_1)
fig1 = plt.figure(figsize=(12,8))
ax1=fig1.add_subplot(111)
sm.graphics.tsa.plot_acf(diff1_144,lags=40,ax=ax1)
fig2 = plt.figure(figsize=(12,8))
ax2=fig2.add_subplot(111)
sm.graphics.tsa.plot_pacf(diff1_144,lags=40, ax=ax2)


arma_mod61 = sm.tsa.ARMA(diff1_144,(6,1)).fit()
print(arma_mod61.aic,arma_mod61.bic,arma_mod61.hqic)
resid = arma_mod61.resid

fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(resid.values.squeeze(), lags=40, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(resid, lags=40, ax=ax2)

print(sm.stats.durbin_watson(arma_mod61.resid.values))

r,q,p = sm.tsa.acf(resid.values.squeeze(), qstat=True)
d = np.c_[range(1,41), r[1:], q, p]
table = pd.DataFrame(d, columns=['lag', "AC", "Q", "Prob(>Q)"])
print(table.set_index('lag'))

# prediction
predict_data=arma_mod61.predict(0,-1,dynamic=False)
# Since the prediction is made with the difference value, the result needs to be restored
# 144 differential restore
diff1_144_shift=diff_1.shift(144)
# print('print diff1_144_shift')
print(diff1_144_shift)
diff_recover_144=predict_data.add(diff1_144_shift)
# First difference reduction
diff1_shift=data.shift(1)
diff_recover_1=diff_recover_144.add(diff1_shift)
diff_recover_1=diff_recover_1.dropna() # The predicted value of the final reduction
print('predicted value')
print(diff_recover_1)

np.save('/content/drive/MyDrive/MSc/dcrnn/data/result/ARIMA_'+str(lane_index)+'.npy', diff_recover_1)